In [ ]:
from __future__ import annotations

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    SimulationResult,
    Transmon,
)

SAMPLING_PERIOD = 0.5
qp.set_sampling_period(SAMPLING_PERIOD)

In [ ]:
# units: ns, GHz
transmon_dimension = 3
control_frequency = 7.456
target_frequency = 8.654
anharmonicity = -0.333
relaxation_rate = 0.00005
dephasing_rate = 0.00005

qubits = [
    Transmon(
        label="Q0",
        dimension=transmon_dimension,
        frequency=control_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
    Transmon(
        label="Q1",
        dimension=transmon_dimension,
        frequency=target_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
]

couplings = [
    Coupling(
        pair=("Q0", "Q1"),
        strength=0.01,
    ),
]

system = QuantumSystem(
    objects=qubits,
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
control_qubit = qubits[0]
target_qubit = qubits[1]

In [ ]:
def drag_pi_pulse(
    duration: float,
    anharmonicity: float,
) -> qp.Pulse:
    alpha = 2 * np.pi * anharmonicity
    pulse = qp.Drag(
        duration=duration,
        amplitude=1,
        beta=-0.5 / alpha,
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    return pulse

In [ ]:
pi_pulse = drag_pi_pulse(
    duration=16,
    anharmonicity=control_qubit.anharmonicity,
)
pi_pulse.plot(divide_by_two_pi=True)

result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit.label,
            frequency=control_qubit.frequency,
            waveform=pi_pulse,
            durations=[SAMPLING_PERIOD] * pi_pulse.length,
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere("Q0")
result.show_last_population("Q0")

In [ ]:
def cr_sequence(
    cr_amplitude: float,
    cr_duration: float,
    cr_ramp: float,
    cr_phase: float,
    crosstalk_amplitude: float = 0.0,
    crosstalk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    echo: bool = False,
):
    cr_waveform = qp.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * cr_amplitude,
        tau=cr_ramp,
        phase_shift=cr_phase,
    )
    cancel_waveform = qp.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * cancel_amplitude,
        tau=cr_ramp,
        phase_shift=cancel_phase,
    )
    crosstalk_waveform = qp.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * crosstalk_amplitude,
        tau=cr_ramp,
        phase_shift=crosstalk_phase,
    )
    with qp.PulseSchedule(
        {
            "CR": {
                "frequency": target_qubit.frequency,
                "object": control_qubit.label,
            },
            "Crosstalk": {
                "frequency": control_qubit.frequency,
                "object": target_qubit.label,
            },
            "Target": {
                "frequency": target_qubit.frequency,
                "object": target_qubit.label,
            },
        }
    ) as cr:
        cr.add("CR", cr_waveform)
        cr.add("Target", cancel_waveform)
        cr.add("Crosstalk", crosstalk_waveform)

    if not echo:
        return cr
    else:
        pi_pulse = drag_pi_pulse(
            duration=16,
            anharmonicity=control_qubit.anharmonicity,
        )
        with qp.PulseSchedule(
            {
                "Control": {
                    "frequency": control_qubit.frequency,
                    "object": control_qubit.label,
                },
                "CR": {
                    "frequency": target_qubit.frequency,
                    "object": control_qubit.label,
                },
                "Crosstalk": {
                    "frequency": control_qubit.frequency,
                    "object": target_qubit.label,
                },
                "Target": {
                    "frequency": target_qubit.frequency,
                    "object": target_qubit.label,
                },
            }
        ) as ecr:
            ecr.call(cr)
            ecr.barrier()
            ecr.add("Control", pi_pulse)
            ecr.barrier()
            ecr.call(cr.scaled(-1))
            ecr.barrier()
            ecr.add("Control", pi_pulse)
        return ecr

In [ ]:
cr_sequence(
    cr_amplitude=0.2,
    cr_duration=200,
    cr_ramp=30.0,
    cr_phase=1.0,
    crosstalk_amplitude=0.2 * 0.01,
    crosstalk_phase=2.0,
    cancel_amplitude=0.02,
    cancel_phase=0.0,
).plot(
    title="Cross Resonance Sequence",
    width=500,
    divide_by_two_pi=True,
)

In [ ]:
cr_sequence(
    cr_amplitude=0.2,
    cr_duration=200,
    cr_ramp=30.0,
    cr_phase=1,
    crosstalk_amplitude=0.2 * 0.01,
    crosstalk_phase=2.0,
    cancel_amplitude=0.02,
    cancel_phase=0.0,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)

In [ ]:
def simulate_cr(
    cr_amplitude: float,
    cr_duration: float,
    cr_ramp: float,
    cr_phase: float,
    crosstalk_amplitude: float = 0.0,
    crosstalk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    echo: bool = False,
    control_state: str = "0",
    n_samples: int = 100,
    plot: bool = False,
) -> SimulationResult:
    controls = cr_sequence(
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramp=cr_ramp,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=echo,
    )
    initial_state = system.state(
        {
            control_qubit.label: control_state,
            target_qubit.label: "0",
        },
    )
    result = simulator.mesolve(
        controls=controls,
        initial_state=initial_state,
        n_samples=n_samples,
    )
    if plot:
        result.plot_bloch_vectors(target_qubit.label)
        result.display_bloch_sphere(target_qubit.label)
    return result

In [ ]:
cr_duration = 1000  # ns
cr_amplitude = 0.2  # GHz
cr_ramp = 30  # ns
cr_phase = 1.0  # rad

crosstalk_amplitude = cr_amplitude * 0.01  # GHz
crosstalk_phase = 2.0  # rad

In [ ]:
result_0 = simulate_cr(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    control_state="0",
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    control_state="1",
    plot=True,
)

In [ ]:
times = result_0.times
vectors_0 = result_0.get_bloch_vectors(target_qubit.label)
vectors_1 = result_1.get_bloch_vectors(target_qubit.label)

In [ ]:
R = np.sqrt(np.sum((vectors_0 + vectors_1) ** 2, axis=1))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
)

for i, component in enumerate(["X", "Y", "Z"]):
    fig.add_trace(
        go.Scatter(
            mode="lines+markers",
            x=times,
            y=vectors_0[:, i],
            name=f"0_{component}",
        ),
        row=i + 1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            mode="lines+markers",
            x=times,
            y=vectors_1[:, i],
            name=f"1_{component}",
        ),
        row=i + 1,
        col=1,
    )

fig.add_trace(
    go.Scatter(
        mode="lines+markers",
        x=times,
        y=R,
        name="R",
    ),
    row=4,
    col=1,
)

fig.update_layout(
    title_text="Bloch vectors",
    showlegend=False,
    height=500,
    yaxis_range=[-1, 1],
)

fig.update_xaxes(title_text="Time (ns)", row=4, col=1)

fig.update_yaxes(title_text="〈X〉", row=1, col=1)
fig.update_yaxes(title_text="〈Y〉", row=2, col=1)
fig.update_yaxes(title_text="〈Z〉", row=3, col=1)
fig.update_yaxes(title_text="|R|", row=4, col=1)

fig.show()

In [ ]:
# Remove the ramp-up and ramp-down parts of the data
indices = (times >= cr_ramp) & (times < times[-1] - cr_ramp)
times_ = times[indices] - cr_ramp * 0.5
vectors_0_ = vectors_0[indices]
vectors_1_ = vectors_1[indices]

In [ ]:
import qctrlvisualizer as qv

from qubex.analysis.fitting import fit_rotation3d

Omega_0 = fit_rotation3d(times_, vectors_0_)
qv.display_bloch_sphere_from_bloch_vectors(vectors_0_)

Omega_1 = fit_rotation3d(times_, vectors_1_)
qv.display_bloch_sphere_from_bloch_vectors(vectors_1_)

In [ ]:
Omega = np.concatenate(
    [
        0.5 * (Omega_0 + Omega_1),
        0.5 * (Omega_0 - Omega_1),
    ]
)

coeffs = dict(
    zip(
        ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
        0.5 * Omega / (2 * np.pi),
    )
)

for key, value in coeffs.items():
    print(f"{key}: {value * 1e3:+.6f} MHz")

In [ ]:
def hamiltonian_tomography(
    cr_duration: float,
    cr_amplitude: float,
    cr_ramp: float,
    cr_phase: float,
    cross_talk_amplitude: float = 0.0,
    cross_talk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    n_samples: int = 100,
    plot: bool = False,
) -> dict:
    result_0 = simulate_cr(
        cr_duration=cr_duration,
        cr_amplitude=cr_amplitude,
        cr_ramp=cr_ramp,
        cr_phase=cr_phase,
        crosstalk_amplitude=cross_talk_amplitude,
        crosstalk_phase=cross_talk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        control_state="0",
        n_samples=n_samples,
        plot=plot,
    )
    result_1 = simulate_cr(
        cr_duration=cr_duration,
        cr_amplitude=cr_amplitude,
        cr_ramp=cr_ramp,
        cr_phase=cr_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        control_state="1",
        n_samples=n_samples,
        plot=plot,
    )

    times = result_0.times
    vectors_0 = result_0.get_bloch_vectors(target_qubit.label)
    vectors_1 = result_1.get_bloch_vectors(target_qubit.label)

    indices = (times >= cr_ramp) & (times < times[-1] - cr_ramp)
    times_ = times[indices] - cr_ramp * 0.5
    vectors_0_ = vectors_0[indices]
    vectors_1_ = vectors_1[indices]

    Omega_0 = fit_rotation3d(times_, vectors_0_, plot=plot)
    Omega_1 = fit_rotation3d(times_, vectors_1_, plot=plot)
    Omega = np.concatenate(
        [
            0.5 * (Omega_0 + Omega_1),
            0.5 * (Omega_0 - Omega_1),
        ]
    )

    coeffs = dict(
        zip(
            ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
            0.5 * Omega / (2 * np.pi),
        )
    )

    for key, value in coeffs.items():
        print(f"{key}: {value * 1e3:+.6f} MHz")

    return {
        "Omega": Omega,
        "coeffs": coeffs,
        "cr_phase_est": np.arctan2(coeffs["ZY"], coeffs["ZX"]),
        "cancel_amplitude_est": -coeffs["IX"] * 2,
    }

In [ ]:
result_0 = hamiltonian_tomography(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    plot=True,
)

In [ ]:
cr_phase_est = result_0["cr_phase_est"]
cr_phase_est

In [ ]:
result_1 = hamiltonian_tomography(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase - cr_phase_est,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    plot=True,
)

In [ ]:
cancel_amplitude_est = result_1["cancel_amplitude_est"]
cancel_amplitude_est

In [ ]:
result_2 = hamiltonian_tomography(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase - cr_phase_est,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=0.0,
    plot=True,
)

In [ ]:
cr_sequence(
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase - cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=0.0,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)

In [ ]:
result_ecr_0 = simulate_cr(
    cr_duration=275,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase - cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=0.0,
    control_state="0",
    echo=True,
    plot=True,
)

In [ ]:
result_ecr_1 = simulate_cr(
    cr_duration=275,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase - cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=0.0,
    control_state="1",
    echo=True,
    plot=True,
)